In [ ]:
print("Please upvote :)")

In [ ]:
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
import numpy as np
from sklearn.utils import shuffle
import os

In [ ]:
directory = "../input/happywhale-resized-images/happywhale_resized_images"
filenames = os.listdir(directory)
filenames = shuffle(filenames, random_state=0)
filenames[:5], len(filenames)

In [ ]:
filepath = "../input/happy-whale-and-dolphin/train.csv"
train = pd.read_csv(filepath)
train

In [ ]:
sorted_unique_species = sorted(train.species.unique())
train.species = train.species.apply(lambda x: sorted_unique_species.index(x))
train

In [ ]:
def bytes_feature(value):
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [ ]:
split = np.linspace(0, len(filenames), 23, dtype=int)

for i in tqdm(range(len(split) - 1)):
    with tf.io.TFRecordWriter(f"classification_{i}.tfrecords") as writer:
        for j in range(split[i], split[i + 1]):
            filename = filenames[i]
            filepath = os.path.join(directory, filename)
            image_bytes = tf.io.read_file(filepath)
            image = tf.io.decode_jpeg(image_bytes)
            image_bytes = bytes(image)
            species = train.species[train.image == filename].values[0]
            species = np.array([species], "int32")
            species_bytes = bytes(species)
            feature = {
                "input": bytes_feature(image_bytes),
                "output": bytes_feature(species_bytes),
            }
            tf_example = tf.train.Example(features=tf.train.Features(feature=feature))
            writer.write(tf_example.SerializeToString())